In [ ]:
import sys
import time

In [ ]:
import numpy as np
import pandas as pd
#import jax.numpy as np
#import jaxopt

In [ ]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.dummy import DummyClassifier

from sklearn.multioutput import MultiOutputClassifier, ClassifierChain

from sklearn.model_selection import train_test_split

from sklearn.random_projection import GaussianRandomProjection

from sklearn.datasets import load_svmlight_file
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from scipy.special import expit
from scipy.optimize import minimize as sp_minimize

---
## Loading dataset

In [ ]:
dataset_name='yeast'
#dataset_name='scene'
#dataset_name='tmc2007'

In [ ]:
X_train, y_train_ = load_svmlight_file(dataset_name+'_train.svm', multilabel=True)
X_train = np.array(X_train.todense())
X_train.shape

In [ ]:
X_test, y_test_ = load_svmlight_file(dataset_name+'_test.svm', multilabel=True)
X_test = np.array(X_test.todense())
X_test.shape

In [ ]:
onehot_labeller = MultiLabelBinarizer()
y_train = onehot_labeller.fit_transform(y_train_).astype(int)
y_test = onehot_labeller.transform(y_test_).astype(int)

In [ ]:
labels = onehot_labeller.classes_.astype(int)
labels

In [ ]:
plt.hist(y_train.sum(axis=1))

FYI: Error rate of null policy (always predict 0)

In [ ]:
y_test.sum()/(y_test.shape[0])

---

In [ ]:
def micro_hammingloss(p,y):
    assert p.shape == y.shape
    pos = np.where( (p != y) & (y > 0) )
    neg = np.where( (p != y) & (y == 0) )
    fn = p[neg].sum()
    fp = (1-p[pos]).sum()
    return (fn+fp)/(p.shape[0])

In [ ]:
def macro_hammingloss(test_probas, y_test):
    return np.mean([
        micro_hammingloss(test_probas[:,k].reshape((len(y_test),1)), 
                          y_test[:,k].reshape((len(y_test),1))) 
        for k in range(y_test.shape[1])
    ])

---
## Our Model

### CRM routines

In [ ]:
def generate_crm_dataset(X, y, probas, n_samples=4, labels=labels):
    
    assert len(X) == len(y) == len(probas), (len(X) , len(y) , len(probas))
    
    P = []
    A = []
    F = []
    R = []
    
    for i in range(len(X)):
        
        for k in range(n_samples):
            
            chosen_actions = [np.random.binomial(1, p=probas[i,j]) for j in labels]
            chosen_actions_idx = [j for j in labels if chosen_actions[j] > 0]
            chosen_actions_probas = [probas[i,j] for j in labels]
            
#             print(probas[i,:])
#             print(chosen_actions_idx)
#             print(chosen_actions_probas)
            
            A += [np.array(chosen_actions)]
            P += [np.array(chosen_actions_probas)]
            
            x = X[i,:]
            F += [x]

            R += [np.array([ y[i,j]*chosen_actions[j] for j in labels ])]  # how do we scale that with chosen nber of actions ?
#             print(R[-1])
            
    assert len(P) == len(X) * n_samples

    return P, A, R, F

### Modeling

In [ ]:
def model_predict(beta, features):
    # beta is (k, d)
    # features is (n, d)
    p = np.dot(features, beta)
    p = expit(p)
    # predictions is (n,k)
    return p

In [ ]:
def iterate_model(beta, X, y, sampling_probas, prior_crm_dataset, samples_per_instance=4):
    
    # beta should be (k,d)
    
    assert beta.shape == (X.shape[1], y.shape[1])
    
    P, A, R, F = prior_crm_dataset
       
    newP, newA, newR, newF = generate_crm_dataset(
        X, y, sampling_probas, n_samples=samples_per_instance
    )
    assert len(newP) == len(X)*samples_per_instance, (len(newP), len(X), samples_per_instance)
    
    # in form of list for easier stacking through different calls to iterate_model()
    PP = P+newP 
    AA = A+newA
    RR = R+newR
    FF = F+newF
    
    # in form of arrays for model_predict()
    P = np.array(PP) # (n,k) x [0,1] x R
    A = np.array(AA) # (n,k) x {0,1}
    R = np.vstack(RR) # (n,k) x {0,1}
    F = np.vstack(FF) # (n,d) x [0,1] x R
    
    assert len(P) == len(A) == len(R) == len(F), (len(P), len(A), len(R), len(F))
    assert P.shape[1] == A.shape[1] == R.shape[1]
     
    def crm_loss(beta):
        beta = beta.reshape((X.shape[1], y.shape[1]))  # sp.minimize flattens the beta matrix
        pred = model_predict(beta, F)
        W = pred / P
        l = np.sum((1-R)*W) / np.sum(W)
        return l
    
    print('CRM iteration on %d samples - loss: %.4f -> ' % (len(PP), crm_loss(beta)), end='', file=sys.stderr)
    
    solution = sp_minimize(crm_loss, beta, method='L-BFGS-B')
    newbeta = solution.x
    
    final_loss = crm_loss(newbeta)
    
    print('%.4f' % final_loss, file=sys.stderr)
    
    return newbeta.reshape((X.shape[1], y.shape[1])), (PP, AA, RR, FF), final_loss

## Evaluation

In [ ]:
def evaluate_model(beta, X_test, y_test, normalize=False, binarize=True):
    beta_test_probas = model_predict(beta, X_test)
    if normalize:
        beta_test_probas /= beta_test_probas.sum(axis=1).reshape((len(y_test),1))
    if binarize:
        beta_test_probas = (beta_test_probas > .5).astype(int)
    return micro_hammingloss(beta_test_probas, y_test)

----
## Baselines & Skylines

 ![Perf from CRM article](./basesky.png)

In [ ]:
if dataset_name == 'tmc_2007':
    print("reducing dimension for TMC dataset")
    fh = GaussianRandomProjection(n_components=1000)
    X_train = fh.fit_transform(X_train)
    X_test = fh.transform(X_test)
    print(X_train.shape)

In [ ]:
print("pi_null micro test loss:", micro_hammingloss(np.zeros(y_test.shape), y_test))

In [ ]:
pi_dummy = MultiOutputClassifier(DummyClassifier())
pi_dummy.fit(X_train, y_train)

print("pi_dummy train loss:", micro_hammingloss(pi_dummy.predict(X_train), y_train))
print("pi_dummy test loss:", micro_hammingloss(pi_dummy.predict(X_test), y_test))

In [ ]:
pi0 = MultiOutputClassifier(LogisticRegression(), n_jobs=6)

X_0, X_, y_0, y_ = train_test_split(X_train, y_train, test_size=.95, random_state=42)
print('learning pi0 on', len(X_0), 'data points')
pi0.fit(X_0, y_0)

print("pi0 train loss:", micro_hammingloss(pi0.predict(X_train), y_train))
l0 = micro_hammingloss(pi0.predict(X_test), y_test)
print("pi0 test loss:", l0)

In [ ]:
for i in range(len(pi0.estimators_)):
    pi0.estimators_[i].coef_ += .5

In [ ]:
l0 = micro_hammingloss(pi0.predict(X_test), y_test)
l0

In [ ]:
pistar = MultiOutputClassifier(LogisticRegressionCV(max_iter=10000, n_jobs=6))
pistar.fit(X_train, y_train)

In [ ]:
print("pi* train loss:", micro_hammingloss(pistar.predict(X_train), y_train))
lstar = micro_hammingloss(pistar.predict(X_test), y_test)
print("pi* test loss:", lstar)

---
## Sequential CRM

In [ ]:
beta_init = np.random.normal(size=(X_train.shape[1], len(labels)))
print('beta0 H. loss:', evaluate_model(beta_init, X_test, y_test))

In [ ]:
beta_static = np.array(beta_init.copy())
beta_dynamic = np.array(beta_init.copy())

static_crm_dataset = ([],[],[],[])
dynamic_crm_dataset = ([],[],[],[])

n_episods = 10
batch = int(len(X_train) / n_episods)

t_end = t_start = time.time()
for episod in range(n_episods):
    t_end = time.time()
    
    start = episod*batch
    end = (episod+1)*batch
    print('*'*10, 
          'episod: %d/%d' % (episod+1, n_episods), 
          'time: %ds' % (t_end - t_start), 
          '*'*10,
          file=sys.stderr)
    
    t_start = time.time()
    
    # current slice of dataset
    X = X_train[start:end,:]
    y = y_train[start:end,:]
    
    # generating CRM counter-part of current slice
    sampling_probas_static = pi0.predict_proba(X)
    sampling_probas_static = np.array([_[:,1] for _ in sampling_probas_static]).T
    if episod == 0:
        sampling_probas_dynamic = sampling_probas_static
    else:
        sampling_probas_dynamic = model_predict(beta_dynamic, X)

    # optimizing models & evaluate
    ## static CRM
    print('** static policy', file=sys.stderr) 
    beta_static, static_crm_dataset, static_crm_loss = iterate_model(
        beta_static, X, y, sampling_probas_static, static_crm_dataset
    )
    l_stat = evaluate_model(beta_static, X_test, y_test)
    print('H. loss: %.5f (vs pi0: %d%% vs pi*: %d%%)' % (l_stat, 100*l_stat/l0, 100*l_stat/lstar), 
          '|beta|=%.4f' % np.sqrt((beta_static**2).sum()), 
          file=sys.stderr)
    ## sequential CRM
    print('** dynamic policy', file=sys.stderr) 
    beta_dynamic, dynamic_crm_dataset, dynamic_crm_loss = iterate_model(
        beta_dynamic, X, y, sampling_probas_dynamic, dynamic_crm_dataset
    )
    l_dyn = evaluate_model(beta_dynamic, X_test, y_test)
    print('H. loss: %.5f (vs pi0: %d%% vs pi*: %d%%)' % (l_dyn, 100*l_dyn/l0, 100*l_dyn/lstar), 
          '|beta|=%.4f' % np.sqrt((beta_dynamic**2).sum()), 
          file=sys.stderr)
    print(file=sys.stderr)


In [ ]:
plt.title('Sampling Probas per Action')
plt.plot(sampling_probas_static.mean(axis=0),'--', label='static')
plt.plot(sampling_probas_dynamic.mean(axis=0),'--', label='dynamic')
plt.plot(y_test.mean(axis=0), label='label average', alpha=.5)
plt.ylim(0,1)
plt.legend()